# Planetary Motion

This little simulation was inspired by hearing that if we want to travel to mars, there is a special window every 2 years or so that makes it easier. So I wanted to understand why that is. One could use the orbital ellipses and simply let the planets follow them to simulate how the planets move in the solar system. But I was reading Newtons Principia and wanted to use Newton's Law of Universal Gravitation for my simulation.

Estimates of planet data is available at [NASA](https://nssdc.gsfc.nasa.gov/planetary/planetfact.html). However, for someone not fluent in astronomy, it can be quite tricky to interpret those fact sheets. For example; what does the angle 'Longitude of perihelion' represent? Turns out its a sum of two different angles that point in different directions... also perihelion is just a special word for periapsis when the thing it rotates around happens to be a star. The following picture from wikipedia greatly helped in interpreting the fact sheets.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Orbit1.svg/800px-Orbit1.svg.png" width="600" height="400">

I decided to use the following variables to construct a position and velocity of all planets when they are at their closest distance from the sun. It seems like an aesthetically pleasing starting point for a simulation.

1. (m) Mass
2. (pd) Periapsis distance (listed as perihelion distance)
3. (pv) Periapsis velocity (listed as Maximum Velocity)
4. (incl) Orbital inclination
5. (loan) Longitude of ascending node
6. (lop) Longitude of periapsis (listed as longitude of perihelion)

(aop) Argument of periapsis = Longitude of periapsis - Longitude of ascending node

In [1]:
function angles2rotm(a, b, c)
    #construct a rotation matrix from orbital elements (loan, incl, aop)
    # In particular, this is what by convention is called the Z(a) X(b) Z(c) rotation matrix
    # because it rotates around z-axis first, then around the x-axis, then around the z-axis again.
    # each time bringing the coordinate system with it just like in the picture above.
    R=zeros(3,3)
    R[:,1] = [cos(a)*cos(c) - sin(a)*cos(b)*sin(c), -cos(a)*sin(c) - sin(a)*cos(b)*cos(c), sin(a)*sin(b)]
    R[:,2] = [sin(a)*cos(c) + cos(a)*cos(b)*sin(c), -sin(a)*sin(c) + cos(a)*cos(b)*cos(c), -cos(a)*sin(b)]
    R[:,3] = [sin(b)*sin(c), sin(b)*cos(c), cos(b)]
    return R
end

function place_planets_at_periapsis(pd, pv, loan, incl, aop)
    p = zeros(9,3)
    v = zeros(9,3)
    for n=1:9
        R = angles2rotm(loan[n], incl[n], aop[n])
        p[n,:] = R*[pd[n],0,0]
        v[n,:] = R*[0,pv[n],0]
    end
    return p,v
end

place_planets_at_periapsis (generic function with 1 method)

In [2]:
function gravityforce(m, p, b, a)
    d = p[b,:]-p[a,:]
    F = -d .* 6.674e-11*m[a]*m[b]*(d'*d)^(-3/2)
    return F
end

function simulate_solar_system(p, v, m, Ndays) 
    F = zeros(9,3)
    Δt = 24*60*60
    dist_earth_to_planet = zeros(Ndays, 9)

    for t=1:Ndays
        for planet=2:9
            F[planet,:] = gravityforce(m, p, planet, 1)
        end
        a=F./m
        v += a.*Δt
        p += v.*Δt
        
        for planet=1:9
            dist_earth_to_planet[t, planet] = norm(p[planet,:]-p[4,:])/1e9
        end
    end
    return dist_earth_to_planet
end

simulate_solar_system (generic function with 1 method)

In [3]:
m = [1.9885e30, 0.33011e24, 4.8675e24, 5.9724e24, 0.64171e24, 1898.19e24, 568.34e24, 86.813e24, 102.413e24]
pd = [0, 46.00e9, 107.48e9, 147.09e9, 206.62e9, 740.52e9, 1352.55e9, 2741.30e9, 4444.45e9]
pv = [0, 58.98e3, 35.26e3, 30.29e3, 26.50e3, 13.72e3, 10.18e3, 7.11e3, 5.50e3]
incl = π/180*[0, 7.00487, 3.39471, 0.00005, 1.85061, 1.30530, 2.48446, 0.76986, 1.76917]
loan = π/180*[0, 48.33167, 76.68069, -11.26064, 49.57854, 100.55615, 113.71504, 74.22988, 131.72169]
lop  = π/180*[0, 77.45645, 131.53298, 102.94719, 336.04084, 14.75385, 92.43194, 170.96424, 44.97135];
aop = lop - loan;

p, v = place_planets_at_periapsis(pd, pv, loan, incl, aop)
dist_earth_to_planet = simulate_solar_system(p, v, m, 365*100);

In [4]:
function avgperiod(e2p)
    binaray_toward_away = e2p[2:end].>e2p[1:end-1]
    index_of_minima = find(binaray_toward_away[2:end].>binaray_toward_away[1:end-1])
    #periods = index_of_minima[2:end].-index_of_minima[1:end-1]
    #avgperiod = mean(periods[2:end-1])
    meanperiod = mean(index_of_minima[2:end].-index_of_minima[1:end-1])
    return meanperiod
end

names=["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune"]
println("Average period of Earth close approach with")
for planet=[1,2,3,5,6,7,8,9]
    period = avgperiod(dist_earth_to_planet[:,planet])
    println(names[planet], ": ", Int(round(period)), " days")
    #println(names[planet], ": ", round(period,1), " days")
end

Average period of Earth close approach with
Sun: 365 days
Mercury: 117 days
Venus: 585 days
Mars: 780 days
Jupiter: 399 days
Saturn: 378 days
Uranus: 370 days
Neptune: 368 days


# Visualization

In [ ]:
using Plots; gr(size=(600,750))
plot(leg=false, xtick=false, ytick=false, axis=false)

function simulate_solar_system_VISUALIZE(p, v, m, Ndays) 
    F = zeros(9,3)
    Δt = 24*60*60
    ss = norm(p[5,:])*1.5 #size of plot
    
    @gif for t=1:Ndays
        for n=2:9; F[n,:] = gravityforce(m, p, n, 1) end
        a=F./m
        v += a.*Δt
        p += v.*Δt

        # visualize
        if t%5==0
            p1 = scatter(p[:,1],p[:,2], xlim=(-ss,ss), ylim=(-ss,ss), leg=false, xtick=false, ytick=false, title="Inner Solar System", axis=false)
            plot!(p1, [p[4,1];p[5,1]], [p[4,2];p[5,2]]) # add a line between earth and mars
            p2 = plot(dist_earth_to_planet[1:t,5], ylim=(2e10/1e9, 5e11/1e9), title="Distance Earth-Mars (in million kilometers)", xlab="days")
            plot(p1,p2, layout=grid(2,1,heights=[0.8,0.2]), leg=false)
        end

    end every 5
end

p, v = place_planets_at_periapsis(pd, pv, loan, incl, aop)
simulate_solar_system_VISUALIZE(p, v, m, 365*10);

<img src="tmp.gif">

#### A note on the animation; although the simulatation is in 3 dimensions, the plot is looking straight down on the solarsystem making it look 2 dimensional. However, the plotted earth-mars distance is the correct distance in 3-dimensional space.

# Not all close encounters are equal
As seen in the animation above, there is some funky-ness going on in the earth-mars distance. This comes from the fact that the orbits are not perfectly circular and not perfectly aligned in the same place etc. We have already calculated that the average period of close encounters with mars is about 780 days. But some close encounters are closer than others. We can calculate unusually close... close encounters.

In [ ]:
gr(size=(800,1600))
L=20000
p1=plot(dist_earth_to_planet[1:L,1], title=names[1])
p2=plot(dist_earth_to_planet[1:L,2], title=names[2])
p3=plot(dist_earth_to_planet[1:L,3], title=names[3])
p4=plot(dist_earth_to_planet[1:L,5], title=names[5])
p5=plot(dist_earth_to_planet[1:L,6], title=names[6])
p6=plot(dist_earth_to_planet[1:L,7], title=names[7])
p7=plot(dist_earth_to_planet[1:L,8], title=names[8])
p8=plot(dist_earth_to_planet[1:L,9], title=names[9])
#plot(p1,p2,p3,p4,p5,p6,p7,p8, layout=grid(8,1,heights=[0.1,0.1,0,1.0,1,0.1,0.1,0,1.0,1]), leg=false)
#plot(p1,p2,p3, layout=grid(3,1, heights=[0.1,0.1,0.1]), leg=false)
plot(p1,p2,p3,p4,p5,p6,p7,p8, layout=grid(8,1), leg=false)
#plot(p1,p2)

In [ ]:
p, v = place_planets_at_periapsis(pd, pv, loan, incl, aop)
dist_earth_to_planet = simulate_solar_system(p, v, m, 1000000) #run longer to capture unusually close encounters with outer planets

function avgperiod2(e2p)
    binary_toward_away = e2p[2:end].>e2p[1:end-1]
    index_of_minima = find(binary_toward_away[2:end].>binary_toward_away[1:end-1])
    dist_of_minima=e2p[index_of_minima]

    binary_toward_away2 = dist_of_minima[2:end].>dist_of_minima[1:end-1]
    index_of_minima2 = find(binary_toward_away2[2:end].>binary_toward_away2[1:end-1])

    acc_index = index_of_minima[index_of_minima2]
    meanperiod2 = mean(acc_index[2:end]-acc_index[1:end-1])
end

println("Average period of Earth UNUSUALLY close approach with")
for planet=[2,3,5,6,7,8,9]
    period = avgperiod2(dist_earth_to_planet[:,planet])
    println(names[planet], ": ", Int(round(period)), " days")
    #println(names[planet], ": ", round(period,1), " days")
end

# Conclustions
If we want to get on a spaceship and travel to Mars, there is a window approximately every 2.14 years when mars is close. In addition to that, there is an unusually good window every 15.6 years.

//Anders